# 1DCNN advanced model

In this notebook we trained the 1DCNN advanced model with Keras Tuner using the training and validation set created during preprocessing.

## Initialization

On Colab upload manually the file `dataset.zip`

In [6]:
!unzip -o dataset.zip

Archive:  dataset.zip
  inflating: x_train.npy             
  inflating: y_train.npy             
  inflating: x_train_ov.npy          
  inflating: y_train_ov.npy          
  inflating: x_val.npy               
  inflating: y_val.npy               


### Import libraries

In [7]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16) 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import sklearn.preprocessing as sklp
import warnings
import logging

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.9.2


In [8]:
!pip install -q -U keras-tuner
import keras_tuner as kt

### Set seed for reproducibility

In [9]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Load dataset

In [10]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_val = np.load("x_val.npy")
y_val = np.load("y_val.npy")

x_train.shape, x_val.shape, y_train.shape, y_val.shape

((1938, 36, 6), (491, 36, 6), (1938, 12), (491, 12))

## Model

In [11]:
input_shape = (36, 6)
classes = 12
batch_size = 32
epochs = 100

### Custom Augmentation Layer

In [22]:
class CustomAugmentationLayer(tfkl.Layer):
    def __init__(self, **kwargs):
        super(CustomAugmentationLayer, self).__init__(**kwargs)

    def random_float(self, start, end, precision):
      return round(random.uniform(start, end), precision)

    def custom_scaling(self, x):
      return x * (1 - self.random_float(0, 0.2, 2))

    def custom_shift(self, x):
      return x + self.random_float(-0.2, 0.2, 2)

    def call(self, inputs, training=None):
        if training:
  
          if random.random() < 0.5:
            x = self.custom_scaling(inputs)

          if random.random() < 0.5:
            x = self.custom_shift(inputs)

        return inputs

### Definition

In [32]:
def build_model(hp):
  model = tfk.models.Sequential(name='1DCNN')
  model.add(tfkl.Input(input_shape, name='input_layer'))

  model.add(tfkl.Normalization())
  model.add(CustomAugmentationLayer(name='custom_augmentation_layer'))

  # First convolutional layer
  hp_filters_1 = hp.Int('filters_1', min_value=64, max_value=1024, step=64)
  hp_kernels_1 = hp.Int('kernels_1', min_value=3, max_value=8, step=1)
  model.add(tfkl.Conv1D(filters=hp_filters_1, kernel_size=hp_kernels_1, padding="same", name='conv_1'))
  model.add(tfkl.BatchNormalization())
  model.add(tfkl.ReLU())

  # Second convolutional layer
  hp_filters_2 = hp.Int('filters_2', min_value=64, max_value=1024, step=64)
  hp_kernels_2 = hp.Int('kernels_2', min_value=3, max_value=8, step=1)
  model.add(tfkl.Conv1D(filters=hp_filters_2, kernel_size=hp_kernels_2, padding="same", name='conv_2'))
  model.add(tfkl.BatchNormalization())
  model.add(tfkl.ReLU())

  # Third convolutional layer
  hp_filters_3 = hp.Int('filters_3', min_value=64, max_value=1024, step=64)
  hp_kernels_3 = hp.Int('kernels_3', min_value=3, max_value=8, step=1)
  model.add(tfkl.Conv1D(filters=hp_filters_3, kernel_size=hp_kernels_3, padding="same", name='conv_3'))
  model.add(tfkl.BatchNormalization())
  model.add(tfkl.ReLU())

  # Fourth convolutional layer
  hp_filters_4 = hp.Int('filters_4', min_value=64, max_value=1024, step=64)
  hp_kernels_4 = hp.Int('kernels_4', min_value=3, max_value=8, step=1)
  model.add(tfkl.Conv1D(filters=hp_filters_4, kernel_size=hp_kernels_4, padding="same", name='conv_4'))
  model.add(tfkl.BatchNormalization())
  model.add(tfkl.ReLU())

  # Gap
  model.add(tfkl.GlobalAveragePooling1D())
  hp_gap_dropout = hp.Float('gap_dropout', min_value=0, max_value=0.4, step=0.1)
  model.add(tfkl.Dropout(hp_gap_dropout, seed=seed, name='conv_gap_dropout'))

  # First dense layer
  hp_dense_1_units = hp.Int('dense_1_units', min_value=64, max_value=512, step=32)
  model.add(tfkl.Dense(hp_dense_1_units, activation='relu', name='conv_dense_1'))

  # Second dense layer
  hp_dense_2_units = hp.Int('dense_2_units', min_value=64, max_value=512, step=32)
  model.add(tfkl.Dense(hp_dense_2_units, activation='relu', name='conv_dense_2'))

  hp_final_dropout = hp.Float('final_dropout', min_value=0, max_value=0.4, step=0.1)
  model.add(tfkl.Dropout(hp_final_dropout, seed=seed, name='conv_final_dropout'))

  # Output
  model.add(tfkl.Dense(classes, activation="softmax", name="conv_output"))

  # Compile
  model.compile(optimizer=tfk.optimizers.Adam(), loss=tfk.losses.CategoricalCrossentropy(), metrics=['accuracy'])

  return model

### Tuner

In [33]:
!rm -rf tuner

In [34]:
tuner = kt.Hyperband(
  build_model,
  objective='val_loss',
  max_epochs=epochs+20,
  factor=3,
  seed=seed,
  directory='tuner',
  project_name='tuner'
)

In [35]:
# Search for the best model
tuner.search(
  x=x_train, 
  y=y_train, 
  epochs=epochs, 
  batch_size=batch_size, 
  validation_data=(x_val, y_val),
  callbacks = [
    tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.1, min_lr=1e-5),
    tfk.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
  ]
)

# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]

# Get the best model
best_model = tuner.get_best_models()[0]

# Print summary of best model
tuner.results_summary(1)

Trial 254 Complete [00h 01m 02s]
val_loss: 1.2056190967559814

Best val_loss So Far: 1.1142886877059937
Total elapsed time: 01h 18m 15s
Results summary
Results in tuner/tuner
Showing 1 best trials
Trial summary
Hyperparameters:
filters_1: 640
kernels_1: 6
filters_2: 640
kernels_2: 7
filters_3: 384
kernels_3: 5
filters_4: 320
kernels_4: 7
gap_dropout: 0.2
dense_1_units: 160
dense_2_units: 448
final_dropout: 0.1
tuner/epochs: 120
tuner/initial_epoch: 40
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0233
Score: 1.1142886877059937


### Results

In [36]:
train_loss, train_accuracy = best_model.evaluate(x_train, y_train)
val_loss, val_accuracy = best_model.evaluate(x_val, y_val)
print("train_loss:  %.4f  train_accuracy: %.4f" % (train_loss, train_accuracy))
print("val_loss:    %.4f  val_accuracy:   %.4f" % (val_loss, val_accuracy))

16/16 [==============================] - 0s 7ms/step - loss: 1.1143 - accuracy: 0.6253
train_loss:  0.8467  train_accuracy: 0.6873
val_loss:    1.1143  val_accuracy:   0.6253


### Save

In [37]:
best_model.save("model")
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/saved_model.pb (deflated 89%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: model/variables/variables.index (deflated 72%)
  adding: model/assets/ (stored 0%)
  adding: model/keras_metadata.pb (deflated 93%)
